In [1]:
import os
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

# 환경 변수에서 API 키 불러오기
api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
#print(api_key) 확인

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

# 모델 초기화
model = ChatOpenAI(model="gpt-4o-mini")

In [3]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "인공지능최신동향.pdf"
# PDF 파일 경로
loader = PyPDFLoader(file_path=file_path)

docs = loader.load()


In [4]:
page_number = docs[1].metadata['page']
print(page_number)
page_text = docs[0].page_content
print(page_text)
print(list(docs[0].metadata.keys())) # matadata의 key가 어떤것들이 있는지 확인 가능하다

1
2024년 11월호
['source', 'page']


In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=30,
    length_function=len,
    is_separator_regex=False,
)

splits = recursive_text_splitter.split_documents(docs)

In [6]:
from langchain_openai import OpenAIEmbeddings
from langchain.storage import LocalFileStore
from langchain.embeddings import CacheBackedEmbeddings

# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")


# 로컬 파일 저장소 설정
store = LocalFileStore("C:\\Users\\1\\Desktop\\emb")

# 캐시를 지원하는 임베딩 생성 - 임베딩시 계속 api 호출을 방지하기 위해 로컬에 임베팅 파일을 저장하는 형식
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings=embeddings,
    document_embedding_cache=store,
    namespace=embeddings.model,  # 기본 임베딩과 저장소를 사용하여 캐시 지원 임베딩을 생성
)

In [7]:
import faiss
from langchain_community.vectorstores import FAISS
    
    
vectorstore = FAISS.from_documents(documents=splits, embedding=cached_embedder)

In [8]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3}) # 가져올 청크 수를 3으로 늘림

In [ ]:
# 디버그 및 컨텍스트 처리 클래스

In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableMap

class DebugPassThrough(RunnablePassthrough):
    def invoke(self, *args, **kwargs):
        output = super().invoke(*args, **kwargs)
        print("Debug Output:", output)
        return output
# 문서 리스트를 텍스트로 변환하는 단계 추가
class ContextToText(RunnablePassthrough):
    def invoke(self, inputs, config=None, **kwargs):  # config 인수 추가
        # context의 각 문서를 문자열로 결합
        context_text = "\n".join([doc.page_content for doc in inputs["context"]])
        return {"context": context_text, "question": inputs["question"]}


 프롬프트 파싱 함수 정의하기

In [10]:
def load_chat_prompt_template(prompt_path):
    '''
    프롬프트 경로에서 프롬프트 파일을 읽어 시스템 메시지와 사용자 메시지로 분리하고
    이를 사용하여 ChatPromptTemplate을 생성
    '''
    with open(prompt_path, 'r', encoding='utf-8') as file:
        prompt_text = file.read()
    # 'system'과 'human'으로 분리
    sections = prompt_text.strip().split('human') # 문자열 양쪽에 공백 제거 후, 'human'찾아 섹션을 나누고 리스트로 분리하기 

    # 정의 부분
    system_prompt = '' 
    user_prompt = ''
    if len(sections) == 2: # system과 human으로 분리되어 있을 때
        system_prompt = sections[0].replace('system', '').strip()
        user_prompt = sections[1].strip()
    else:
        user_prompt = prompt_text.strip() # 아니면 바로 user_prompt로 할당.
    # 메시지 리스트 생성
    messages = []
    if system_prompt:
        messages.append(("system", system_prompt))
    if user_prompt:
        messages.append(("user", user_prompt))
    # ChatPromptTemplate 생성
    prompt = ChatPromptTemplate.from_messages(messages)
    return prompt


In [11]:
# 체인 생성 함수
def create_rag_chain(prompt_template, retriever, model):
    chain = (
        RunnableMap({
            "question": DebugPassThrough(),
            "context": retriever
        })
        | ContextToText()
        | prompt_template
        | model
    )
    return chain

In [19]:
from datetime import datetime
# 타임스탬프 생성
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
# 프롬프트 및 응답 폴더 설정
prompt_folder = 'Prompts'
prompt_files = [f for f in os.listdir(prompt_folder) if f.endswith('.txt')]
output_folder = 'responses'
os.makedirs(output_folder, exist_ok=True)

In [17]:
response = chain.invoke(query)
print(type(response.content))  # <class 'dict'>

Debug Output: 주요 행사 일정의 개요를 정리해줘
<class 'str'>


In [20]:
while True:
    print("========================")
    query = input("질문을 입력하세요 (종료하려면 빈 줄 입력): ")
    if not query.strip():
        break
    for prompt_file in prompt_files:
        prompt_path = os.path.join(prompt_folder, prompt_file)
        prompt_template = load_chat_prompt_template(prompt_path)
        chain = create_rag_chain(prompt_template, retriever, model)
        print(f"\nUsing prompt from {prompt_file}")
        response = chain.invoke(query) # 문자열 요구 chain.invoke({"question": query}) -> chain.invoke(query) 수정
        print("Final Response:")
        print(response.content)
        # 응답 저장
        output_file = f"{os.path.splitext(prompt_file)[0]}_{timestamp}_result.txt"
        output_path = os.path.join(output_folder, output_file)
        with open(output_path, 'a', encoding='utf-8') as file:
            file.write("\nQuestion: " + query + "\nResponse: " + response.content + "\n")
        print(f"Response saved to {output_file}")

질문을 입력하세요 (종료하려면 빈 줄 입력):  주요 행사 일정의 개요를 정리해줘



Using prompt from Prompt1.txt
Debug Output: 주요 행사 일정의 개요를 정리해줘
Final Response:
주요 행사 일정은 다음과 같습니다:

1. **NeurIPS 2024**
   - 기간: 2024.12.10~15
   - 장소: 캐나다 밴쿠버
   - 개요: 인공지능과 머신러닝 분야의 연구 성과 교환을 촉진하는 연례 학술대회로, AI 연구자를 위한 실험 설계, LLM을 위한 메타 생성 알고리즘, 정렬에 대한 학제 간 통찰력 등을 다룰 예정.
   - 홈페이지: [neurips.cc](https://neurips.cc/)

2. **GenAI Summit Maroc 2024**
   - 기간: 2024.12.10~11
   - 장소: 모로코
   - 개요: 인공지능과 데이터 분석에 초점을 맞춘 행사로, 250명 이상의 업계 리더, 정책 입안자, 전문가가 모여 AI 발전을 탐구하며, 오픈소스 AI, AI 주도 사이버 보안 등을 다룰 예정.
   - 홈페이지: [genaimaroc.com](https://genaimaroc.com/)

3. **AI Summit Seoul 2024**
   - 기간: 2024.12.10~11
   - 장소: 서울(코엑스 그랜드볼룸)
   - 개요: AI와 산업의 융합에 초점을 두고 다양한 글로벌 기업과 기관, 학계 전문가들이 모여 AI 및 산업 트렌드에 대한 주제 발표 및 워크샵을 진행.
   - 홈페이지: [aisummit.co.kr](https://aisummit.co.kr/)
Response saved to Prompt1_20241115_142347_result.txt

Using prompt from Prompt2.txt
Debug Output: 주요 행사 일정의 개요를 정리해줘
Final Response:
주요 행사 일정은 다음과 같습니다:  
1. NeurIPS 2024 - 2024년 12월 10일부터 15일까지 캐나다 밴쿠버에서 개최.  
2. GenAI Summit Maro

질문을 입력하세요 (종료하려면 빈 줄 입력):   
